# Metaprogramming em Julia

In [4]:
x = 10
y = 5

@show y + x

y + x = 15


15

In [5]:
@macroexpand @show y + x

quote
    Base.println("y + x = ", Base.repr(begin
                #= show.jl:955 =#
                local var"#788#value" = y + x
            end))
    var"#788#value"
end

Veja que a expressão "quote" simboliza que essa expressão é código em Julia. Ou seja, poderíamos copiar e 
colar o que está dentro desse "quote" no nosso REPL. A segunda expressão estranha é esse comentário
"#= show.jl:955 =#". Neste caso, esse comentário está informando em que linha do source code está definida essa função.
Por fim, temos `var"#788#value"`. Isso é o nome da variável que o código criou. Esse nome é estranho
justamente com o propósito de evitar que já tenha essa variável no se código.

In [6]:
code = Meta.parse("j = i^2")

:(j = i ^ 2)

In [7]:
typeof(code)

Expr

In [8]:
dump(code)

Expr
  head: Symbol =
  args: Array{Any}((2,))
    1: Symbol j
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol ^
        2: Symbol i
        3: Int64 2


In [12]:
code.args

2-element Vector{Any}:
 :j
 :(i ^ 2)

In [13]:
code.head

:(=)

In [15]:
dump(code.args[2])

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol ^
    2: Symbol i
    3: Int64 2


In [23]:
code2 = copy(code)
code2.args[1] = :k
code2.args[2].args[3] = 3
code2

:(k = i ^ 3)

Em Julia, temos o tipo `Symbol` que pode ser entendido como uma expressão antes de ser
avaliada. Por exemplo, caso escrevamos `x = 2`. Ao escrever `x`, o compilador
irá avaliar essa expressão, retornando `2`. Entretanto, as vezes queremos
nos referir não ao que está dentro de `x`, mas ao símbolo em si. Para isso,
usamos `:x`.

In [25]:
Symbol("x") == :x

true

Da mesma forma, podemos falar sobre a expressão `x + y` usando `:(x + y)`. Considere a seguinte situação,
você está escrevendo uma série de comandos no REPL. Você percebe que declarou tantas variáveis que não sabe mais
qual delas está armazenando o valor `10`. Julia permite que você faça isso de dentro da própria linguagem.

In [60]:
x = 1
y = 10
z = 2

for i in 'x':'z'
    v = Meta.parse("$i")
    if eval(v) == 10
        return v
    end
end

:y

A função `eval` está justamente avaliando a expressão dentro de `v`. O que fizemos no código acima foi iterar pela
nossas variáveis nos referindo a elas pelo seu símbolo. Imagine agora que cada uma das letras do alfabeto
foi definida como uma variável e queremos descobrir novamente qual delas armazena `10`. Como faríamos
isso sem metaprogramação? Possívelmente teríamos que manualmente escrever algo do tipo
`for v in [a,b,c,...,z]`. Com metaprogramação, podemos simplesmente escrever o loop usando `'a':'z'`.